In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/30 16:03:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer

In [5]:
from pyspark.sql.functions import col, udf

In [6]:
from pyspark.sql.types import IntegerType

In [7]:
sen_df = spark.createDataFrame([
    (0, 'Hi I heard about spark'),
    (1, 'I wish java could use case classes'),
    (2, 'Logestic,regression,model,are,neat'),    
],['Id', 'sentence'])

In [8]:
sen_df.show()

+---+--------------------+
| Id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish java could...|
|  2|Logestic,regressi...|
+---+--------------------+



In [9]:
sen_df.show()

+---+--------------------+
| Id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish java could...|
|  2|Logestic,regressi...|
+---+--------------------+



In [10]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [11]:
reg_tokenizer = RegexTokenizer(inputCol='sentence', outputCol='words', pattern='\\W')

In [12]:
count_token = udf(lambda words:len(words), IntegerType())

In [13]:
tokenized = tokenizer.transform(sen_df)

In [14]:
tokenized.show()

+---+--------------------+--------------------+
| Id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|
|  1|I wish java could...|[i, wish, java, c...|
|  2|Logestic,regressi...|[logestic,regress...|
+---+--------------------+--------------------+



In [15]:
tokenized.withColumn('token', count_token(col('words'))).show()

+---+--------------------+--------------------+-----+
| Id|            sentence|               words|token|
+---+--------------------+--------------------+-----+
|  0|Hi I heard about ...|[hi, i, heard, ab...|    5|
|  1|I wish java could...|[i, wish, java, c...|    7|
|  2|Logestic,regressi...|[logestic,regress...|    1|
+---+--------------------+--------------------+-----+



In [16]:
rg_tokenized = reg_tokenizer.transform(sen_df)

In [17]:
rg_tokenized.withColumn('token', count_token(col('words'))).show()

+---+--------------------+--------------------+-----+
| Id|            sentence|               words|token|
+---+--------------------+--------------------+-----+
|  0|Hi I heard about ...|[hi, i, heard, ab...|    5|
|  1|I wish java could...|[i, wish, java, c...|    7|
|  2|Logestic,regressi...|[logestic, regres...|    5|
+---+--------------------+--------------------+-----+



In [18]:
from pyspark.ml.feature import StopWordsRemover

In [19]:
sentenceDataframe = spark.createDataFrame([
    (0, ['I', 'saw', 'the', 'green' , 'horse']),
    (1, ['Mary', 'had', 'a', 'little', 'lamb'])]
    ,['id', 'tokens'])

In [20]:
sentenceDataframe.show()

+---+--------------------+
| id|              tokens|
+---+--------------------+
|  0|[I, saw, the, gre...|
|  1|[Mary, had, a, li...|
+---+--------------------+



In [21]:
remover = StopWordsRemover(inputCol='tokens', outputCol='filttered')

In [22]:
remover.transform(sentenceDataframe).show()

+---+--------------------+--------------------+
| id|              tokens|           filttered|
+---+--------------------+--------------------+
|  0|[I, saw, the, gre...| [saw, green, horse]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



In [23]:
from pyspark.ml.feature import NGram

wordDataFrame = spark.createDataFrame([
    (0, ["Hi", "I", "heard", "about", "Spark"]),
    (1, ["I", "wish", "Java", "could", "use", "case", "classes"]),
    (2, ["Logistic", "regression", "models", "are", "neat"])
], ["id", "words"])

In [24]:
ngram = NGram(n=2, inputCol='words', outputCol='gram')

In [25]:
ngram.transform(wordDataFrame).select('gram').show(truncate= False)

+------------------------------------------------------------------+
|gram                                                              |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish Java, Java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+



In [26]:
from pyspark.ml.feature import Tokenizer, IDF, HashingTF

In [29]:
sentencedata = spark.createDataFrame([(0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat")
], ["label", "sentence"])

In [30]:
sentencedata.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi I heard about ...|
|  0.0|I wish Java could...|
|  1.0|Logistic regressi...|
+-----+--------------------+



In [31]:
token_Sen = Tokenizer(inputCol='sentence', outputCol='words')

In [32]:
tokenized_sen = token_Sen.transform(sentencedata)

In [37]:
tokenized_sen.withColumn('count', count_token(col('words'))).show(truncate=False)

+-----+-----------------------------------+------------------------------------------+-----+
|label|sentence                           |words                                     |count|
+-----+-----------------------------------+------------------------------------------+-----+
|0.0  |Hi I heard about Spark             |[hi, i, heard, about, spark]              |5    |
|0.0  |I wish Java could use case classes |[i, wish, java, could, use, case, classes]|7    |
|1.0  |Logistic regression models are neat|[logistic, regression, models, are, neat] |5    |
+-----+-----------------------------------+------------------------------------------+-----+



In [38]:
hashing = HashingTF(inputCol='words', outputCol='rawfeatures')

In [39]:
hashing_word = hashing.transform(tokenized_sen)

In [43]:
hashing_word.show()

+-----+--------------------+--------------------+--------------------+
|label|            sentence|               words|         rawfeatures|
+-----+--------------------+--------------------+--------------------+
|  0.0|Hi I heard about ...|[hi, i, heard, ab...|(262144,[18700,19...|
|  0.0|I wish Java could...|[i, wish, java, c...|(262144,[19036,20...|
|  1.0|Logistic regressi...|[logistic, regres...|(262144,[46243,58...|
+-----+--------------------+--------------------+--------------------+



In [40]:
idf = IDF(inputCol='rawfeatures', outputCol='features')

In [44]:
idf_model = idf.fit(hashing_word)

In [45]:
final_sentences = idf_model.transform(hashing_word)

In [50]:
final_sentences.select('label', 'features').show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(262144,[18700,19...|
|  0.0|(262144,[19036,20...|
|  1.0|(262144,[46243,58...|
+-----+--------------------+



23/12/30 16:20:21 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
23/12/30 16:20:21 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
23/12/30 16:20:21 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


In [51]:
from pyspark.ml.feature import CountVectorizer

In [53]:
df = spark.createDataFrame([(0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))
], ["id", "words"])

In [54]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [55]:
countV = CountVectorizer(inputCol='words', outputCol='features', maxDF=2.0, vocabSize=3)

In [56]:
countV_model = countV.fit(df)

In [57]:
countV_result = countV_model.transform(df)

In [59]:
countV_result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

